In [20]:
from adlframework.retrievals.BlobLocalCache import BlobLocalCache
from adlframework.datasource import DataSource
from adlframework.dataentity.arff_de import ARFFDataEntity
# from adlframework.dataentity.audio_de import AudioRecordingDataEntity
from adlframework.experiment import SimpleExperiment
from keras.losses import KLD, MAE
from keras.optimizers import Adadelta, Adam, SGD
from nets import dense_network, dense_network
import pdb
import numpy as np
from keras.callbacks import ModelCheckpoint

### Custom Controllers/Filters

In [13]:
def remove_wavs(entity):
    '''
    Filter out wav and extraneous files
    '''
    return entity.unique_id[-4:] == 'arff'

def split_to_features(sample):
    f, label = sample
    data = np.array([x for x in f[1]][1:])
    return data, label

### Metadata

In [14]:
cache_path = 'local_cache/Segmented_AVEC/'
train_retrieval = BlobLocalCache(cache_path+'wav/train', cache_path+'labels/train')
val_retrieval = BlobLocalCache(cache_path+'wav/val', cache_path+'labels/val')
test_retrieval = BlobLocalCache(cache_path+'wav/test', cache_path+'labels/test')
max_mem = .3

prefilters = [remove_wavs]
controllers = [split_to_features]

Retrieval not named, so won't be cached.
Retrieval not named, so won't be cached.
Retrieval not named, so won't be cached.


### Data Sources

In [15]:
## Creating and splitting datasets
universal_args = {
    'ignore_cache': True,
    'batch_size': 30,
    'verbosity': 3,
    'controllers': controllers,
    'prefilters': prefilters,
    'max_mem_percent': max_mem,
    'workers': 3,
    
}
train_ds = DataSource(train_retrieval, ARFFDataEntity, **universal_args)

val_ds = DataSource(val_retrieval, ARFFDataEntity, **universal_args)

test_ds = DataSource(test_retrieval, ARFFDataEntity, **universal_args)

100%|██████████| 29996/29996 [00:00<00:00, 900048.24it/s]


In [16]:
### Callbacks
callbacks = [ModelCheckpoint('weights/weights.{epoch:02d}-{val_loss:.2f}.hdf5')]

In [17]:
### Network
net = dense_network(input_shape=(989, ),
					target_shape=(3))

### Experiment

In [18]:
model = net.build_model()

Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 989)               3956      
_________________________________________________________________
dense_4 (Dense)              (None, 256)               253440    
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024      
_________________________________________________________________
dense_5 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 195       
_________________________________________________________________
audio_feature_extractor (Act (None, 3)                 0         
Total params: 275,063
Trainable params: 272,573
Non-trainable params: 2,490
___________________________________________________

In [21]:
epochs = 100
train_steps = 10
val_steps = 3
test_bs = 100
loss = MAE
metrics = ['MAE']

## Optimizer hyperparameters
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
optimizer = sgd

In [ ]:
model.compile(optimizer, loss, metrics)

model.fit_generator(train_ds,
                  train_steps,
                  epochs=epochs,
                  callbacks=callbacks,
                  validation_data=val_ds,
                  validation_steps=val_steps,
                  )

model.test_on_batch(test_ds.next(test_bs))

Epoch 1/100
10/10 [==============================] - 0s - loss: 0.1588 - mean_absolute_error: 0.1588 - val_loss: 0.0945 - val_mean_absolute_error: 0.0945
Epoch 2/100
10/10 [==============================] - 0s - loss: 0.0926 - mean_absolute_error: 0.0926 - val_loss: 0.0874 - val_mean_absolute_error: 0.0874
Epoch 3/100
10/10 [==============================] - 0s - loss: 0.0955 - mean_absolute_error: 0.0955 - val_loss: 0.0878 - val_mean_absolute_error: 0.0878
Epoch 4/100
10/10 [==============================] - 0s - loss: 0.0977 - mean_absolute_error: 0.0977 - val_loss: 0.0887 - val_mean_absolute_error: 0.0887
Epoch 5/100
10/10 [==============================] - 0s - loss: 0.0933 - mean_absolute_error: 0.0933 - val_loss: 0.0786 - val_mean_absolute_error: 0.0786
Epoch 6/100
10/10 [==============================] - 0s - loss: 0.0976 - mean_absolute_error: 0.0976 - val_loss: 0.0771 - val_mean_absolute_error: 0.0771
Epoch 7/100
10/10 [==============================] - 0s - loss: 0.0967 - mea